# 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 지수 표기법 변경
pd.options.display.float_format = '{:.3f}'.format

In [3]:
# data load
path = '/Users/luci031/Desktop/Coding/chai/data'
df = pd.read_csv(path+'/chai_proceed.csv')

In [4]:
# 시계열 처리

df['created_at'] = pd.to_datetime(df['created_at'])
df['birthday'] = pd.to_datetime(df['birthday'])
df['sign_up_date'] = pd.to_datetime(df['sign_up_date'])

In [5]:
# unnamed 삭제
df = df.iloc[::,1:]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5623810 entries, 0 to 5623809
Data columns (total 13 columns):
 #   Column           Dtype         
---  ------           -----         
 0   customer_id      int64         
 1   created_at       datetime64[ns]
 2   pre_discount     int64         
 3   post_discount    int64         
 4   cashback_amount  int64         
 5   discount_amount  int64         
 6   total_promotion  int64         
 7   push_permission  bool          
 8   gender           int64         
 9   is_foreigner     bool          
 10  birthday         datetime64[ns]
 11  sign_up_date     datetime64[ns]
 12  merchant_id      int64         
dtypes: bool(2), datetime64[ns](3), int64(8)
memory usage: 482.7 MB


# 데이터 탐색

## 전반적인 데이터 분포

In [7]:
df.head()

,customer_id,created_at,pre_discount,post_discount,cashback_amount,discount_amount,total_promotion,push_permission,gender,is_foreigner,birthday,sign_up_date,merchant_id
0,207317,2020-03-22 10:27:52.498,36236,36236,1087,0,1087,True,1,False,1957-11-05,2019-11-30 16:48:19.339,9
1,95246,2020-03-12 09:41:09.845,12900,12900,387,0,387,True,1,False,1983-10-23,2019-07-09 08:21:14.098,6
2,208074,2020-02-01 08:54:53.246,7500,7500,225,0,225,True,1,False,1975-07-06,2019-08-15 01:02:13.336,9
3,15552,2020-03-06 10:23:21.183,11800,11800,354,0,354,False,0,False,1996-08-21,2019-08-06 02:09:28.712,4
4,511595,2020-03-13 12:00:27.914,29700,29700,891,0,891,True,1,False,1999-01-23,2020-03-12 00:33:46.147,5


In [8]:
# 데이터 분포
df.describe()

,customer_id,pre_discount,post_discount,cashback_amount,discount_amount,total_promotion,gender,merchant_id
count,5623810.000,5623810.000,5623810.000,5623810.000,5623810.000,5623810.000,5623810.000,5623810.000
mean,272331.670,20455.330,19033.951,122.669,1421.380,1544.048,0.724,5.500
std,157247.954,48731.631,47552.056,593.514,1960.640,1979.354,0.447,2.630
min,1.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000
25%,136186.000,7900.000,6900.000,0.000,0.000,542.000,0.000,3.000
50%,272568.000,11900.000,10900.000,0.000,1000.000,1000.000,1.000,6.000
75%,408236.000,19800.000,17920.000,0.000,2000.000,2000.000,1.000,8.000
max,544921.000,1996000.000,1996000.000,97473.000,500000.000,500000.000,1.000,10.000


**Analysis**
- customer_id는 고유값이므로 제외
- pre_discount와 post_discount의 평균값을 비교해봤을 때 평균적으로 1000원 정도 할인이 들어감을 알 수 있다
- 1회에 최대 결제 금액은 1996000원, 약 200만원 가량
- 평균 캐시백 금액은 122.6원, 최대 캐시백 금액은 97473원, 그러나 3사분면까지 캐시백 금액이 0원인 것으로 보아 캐시백이 많이 이뤄지고 있지는 않다
- 평균 할인 금액은 1421.4원, 최대 할인 금액은 500000원, 캐시백보다 활성화되었음을 알 수 있고, 금액 또한 더 크다
- 둘을 합친 total_promotion의 경우 평균 할인 금액은 1421.3원 가량 -> 캐시백과 일반 할인이 동시에 이뤄지는 지 확인 필요
- Gender의 경우 1이 남성, 0이 여성이므로 1에 가까울수록 남성이 많고, 반대의 경우는 여성이 많은 것으로 해석 가능
- Gender의 평균값은 0.724 -> 남성 유저가 여성 유저보다 결제 건수가 많다는 것을 알 수 있다
- merchant_id는 단순 label 이므로 수치 자체로 탐색하기는 어려움

In [9]:
# 데이터 간 상관관계
df.corr()

,customer_id,pre_discount,post_discount,cashback_amount,discount_amount,total_promotion,push_permission,gender,is_foreigner,merchant_id
customer_id,1.000,0.000,0.000,0.000,-0.000,-0.000,0.001,-0.001,0.003,0.000
pre_discount,0.000,1.000,0.999,0.232,0.614,0.678,-0.019,-0.050,-0.002,-0.000
post_discount,0.000,0.999,1.000,0.243,0.588,0.656,-0.017,-0.048,-0.001,-0.000
cashback_amount,0.000,0.232,0.243,1.000,-0.120,0.181,0.009,-0.012,0.009,0.000
discount_amount,-0.000,0.614,0.588,-0.120,1.000,0.955,-0.048,-0.075,-0.019,0.001
total_promotion,-0.000,0.678,0.656,0.181,0.955,1.000,-0.045,-0.078,-0.016,0.001
push_permission,0.001,-0.019,-0.017,0.009,-0.048,-0.045,1.000,0.060,0.066,0.000
gender,-0.001,-0.050,-0.048,-0.012,-0.075,-0.078,0.060,1.000,0.031,0.001
is_foreigner,0.003,-0.002,-0.001,0.009,-0.019,-0.016,0.066,0.031,1.000,-0.000
merchant_id,0.000,-0.000,-0.000,0.000,0.001,0.001,0.000,0.001,-0.000,1.000


**Analysis**
- customer_id, merchant_id는 단순 label이므로 상관관계 분석이 불가능
- pre_discount, post_discount, cashback_amount, discount_amount, total_promotion은 서로 수치를 이용하여 생성된 값이므로 당연히 상관관계가 있음
- push_permission, gender, is_foreigner 등은 단순한 집계 방식으로는 상관관계 분석이 어려움 -> 가설을 세우고 검정하는 단계 필요

**Result**
- 각 cloumn 별 분석 진행
- customer_id를 통한 유저 별 구매 건수 및 금액 조사 필요
- 구매일시 (월별, 주별, 일별, 시간대별 등), 가입일자를 이용한 데이터 분석 필요
- merchant_id를 이용한 산업별 소비 정도 탐색 필요
- gender, push_permission, is_foreigner 이용한 AB 테스트 실험
- push_permission + birthday 활용한 생일 쿠폰 전략
- 각 거래 및 평균적인 할인 비율 조사 필요